<a href="https://colab.research.google.com/github/harsh204016/NLP-in-Tensorflow/blob/master/embedding_on_imdb_dataset_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
print(tf.__version__)

1.14.0


In [0]:
tf.enable_eager_execution()

In [0]:
!pip install -q tensorflow-datasets

In [0]:
import tensorflow_datasets as tfds
imdb,info=tfds.load("imdb_reviews",with_info=True,as_supervised=True)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


In [0]:
import numpy as np
train,test=imdb['train'],imdb['test']

In [0]:
train_sentences=[]
train_labels=[]

test_sentences=[]
test_labels=[]

for i,j in train:
  train_sentences.append(str(i.numpy()))
  train_labels.append(str(j.numpy()))
for i,j in test:
  test_sentences.append(str(i.numpy()))
  test_labels.append(str(j.numpy()))


In [0]:
train_final_labels=np.array(train_labels)
test_final_labels=np.array(test_labels)

In [0]:
vocab_size=10000
embedded_dim=16
trun='post'
max_len=120
oov='<OOV>'

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
token=Tokenizer(num_words=vocab_size,oov_token=oov)
token.fit_on_texts(train_sentences)
word_index=token.word_index
sequence=token.texts_to_sequences(train_sentences)
padded=pad_sequences(sequence,maxlen=max_len,truncating=trun)

test_seq=token.texts_to_sequences(test_sentences)
test_padded=pad_sequences(test_seq,maxlen=max_len)


In [0]:
from keras.models import Sequential
from keras.layers import Embedding,Dense,Flatten
model=Sequential()
model.add(Embedding(vocab_size,embedded_dim,input_length=max_len))
model.add(Flatten())
model.add(Dense(6,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
#this cell will not work fine , because tf.placeholder is not compatible with tf.eager_execution

In [0]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(train_sentences[1])

b as a <OOV> fan of dickens i have <OOV> been disappointed by adaptations of his novels br br although his works presented an extremely accurate re telling of human life at every level in victorian britain throughout them all was a <OOV> thread of humour that could be both playful or sarcastic as the narrative <OOV> in a way he was a literary <OOV> and <OOV> he could be serious and hilarious in the same sentence he <OOV> pride <OOV> arrogance celebrated modesty and <OOV> with loneliness and poverty it may be a clich xc3 xa9 but he was a people's writer br br and it is the comedy that is so often missing from his interpretations at the
b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he wa

In [0]:
model=tf.keras.Sequential([
                           tf.keras.layers.Embedding(vocab_size,embedded_dim,input_length=max_len),
                           tf.keras.layers.Flatten(), #GlobalAveragePooling1D
                           tf.keras.layers.Dense(6,activation='relu'),
                           tf.keras.layers.Dense(1,activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [0]:
num_epochs = 10
model.fit(padded, train_final_labels, epochs=num_epochs, validation_data=(test_padded, test_final_labels))

Train on 25000 samples, validate on 25000 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
25000/25000 [==============================] - 5s 212us/sample - loss: 0.4912 - acc: 0.7463 - val_loss: 0.3482 - val_acc: 0.8473
Epoch 2/10
25000/25000 [==============================] - 5s 187us/sample - loss: 0.2348 - acc: 0.9093 - val_loss: 0.3790 - val_acc: 0.8346
Epoch 3/10
25000/25000 [==============================] - 5s 189us/sample - loss: 0.0833 - acc: 0.9789 - val_loss: 0.4597 - val_acc: 0.8259
Epoch 4/10
25000/25000 [==============================] - 5s 188us/sample - loss: 0.0197 - acc: 0.9978 - val_loss: 0.5358 - val_acc: 0.8264
Epoch 5/10
25000/25000 [==============================] - 5s 191us/sample - loss: 0.0057 - acc: 0.9994 - val_loss: 0.5991 - val_acc: 0.8244
Epoch 6/10
25000/25000 [==============================] - 5s 189us/sample - loss: 0.0019 - acc: 1.0000 - val_loss: 0.6463 - val_acc: 0.8288
Epoch 7/10
25000/25000 [==============================] - 5s 192us/sample - loss: 9.2217e-04 - acc: 1.0000 - val_loss: 0.6883 - val_acc: 0.8290
Epoch 8/10
25000

In [0]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')